```kewords: tensor, 텐서 연산, 미분, Gradient Descent, ...```

In [1]:
# 굳이 할 필요 없지만, 난 하고 싶으니까
import numpy as np

from tensorflow.keras.datasets import mnist

from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import matplotlib.pyplot as plt

# %matplotlib inline

### 1. 신경망과의 첫 만남

MNIST

- 흑백 손글씨 숫자 이미지
- 1980s, 미국 국립표준기술연구소 (National Institute of Standards and Technology, NIST)
- train: 6만 / test: 1만
- 28, 28, 1
- 10개의 범주 (0 ~ 9)
- 딥러닝계의 "hello world"

In [2]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
print(train_images.shape, len(train_labels))

(60000, 28, 28) 60000


In [4]:
train_labels

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [5]:
print(test_images.shape, len(test_labels))

(10000, 28, 28) 10000


In [6]:
test_labels

array([7, 2, 1, ..., 4, 5, 6], dtype=uint8)

#### 데이터 전처리

In [7]:
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255

test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

In [8]:
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

#### network

In [9]:
network = Sequential()
network.add(Dense(512, input_shape=(28 * 28,), activation="relu"))
network.add(Dense(10, activation="softmax"))

In [10]:
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


신경망의 핵십 구성 요소 = 일종의 데이터 처리 필터, 층 (layer)<br />
= 주어진 문제에 더 의미 있는 표현(representation)을 입력된 데이터로부터 추출.

대부분의 딥러닝은 간단한 층을 연결하여 구성되며, 점진적으로 데이터를 정제하는 형태를 띤다.<br />
따라서 데이터 정제 필터(층)가 연속되어 있는 데이터 프로세싱을 위한 여과기 같음.

- 완전 연결(fully connected)된 신경망 층인 Dense
- Softmax, exponential과 구성비를 이용한 class에 대한 확률값을 반환

compile을 위해 필요한 3가지
- 손실 함수 loss function: 옳은 방햐응로 학습될 수 있도록 도움.
- optimizer: 가중치를 update하는 algorithm
- 훈련 과정을 monitoring할 지표

In [11]:
network.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

In [12]:
network.fit(train_images, train_labels, batch_size=128, epochs=5)

Epoch 1/5
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
469/469 [==============================] - 3s 7ms/step - loss: 0.2566 - accuracy: 0.9255
Epoch 2/5
469/469 [==============================] - 3s 7ms/step - loss: 0.1029 - accuracy: 0.9691
Epoch 3/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0674 - accuracy: 0.9798
Epoch 4/5
469/469 [==============================] - 3s 7ms/step - loss: 0.0492 -

In [13]:
test_loss, test_acc = network.evaluate(test_images, test_labels)
print("test accuracy:", test_acc)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
313/313 [==============================] - 0s 1ms/step - loss: 0.0675 - accuracy: 0.9790
test accuracy: 0.9789999723434448


과대적합 (Overfitting): 모델이 훈련 데이터보다 새로운 데이터에서 성능이 낮아지는 경향

### 2. 신경망을 위한 데이터 표현

tensor: 데이터를 위한 container, 임의의 차원 개수를 가지는 행렬의 일반화된 형태
- 차원 = 축, 랭크


1. Scalar (0D tensor): 숫자 하나만 담고 있는 tensor
- 스칼라 (tensor), 0차원 텐서, 0D 텐서
- in numpy, float32, float64 type

In [14]:
x = np.array(12)
x

array(12)

In [15]:
x.ndim

0

2. Vector (1D tensor)
- 벡터, 1D 텐서

In [16]:
x = np.array([12, 3, 6, 14, 7])
x

array([12,  3,  6, 14,  7])

In [17]:
x.ndim

1

3. Matrix (2D tensor)
- 행렬, 2D 텐서
- 2개의 축 (행-row과 열-column)
- 숫자가 채워진 사각 격자

In [19]:
x = np.array([[5, 78, 2, 34, 0], [6, 79, 3, 35, 1], [7, 80, 4, 36, 2]])
x

array([[ 5, 78,  2, 34,  0],
       [ 6, 79,  3, 35,  1],
       [ 7, 80,  4, 36,  2]])

In [20]:
x.ndim

2

4. 3D tensor ~
- 3D 텐서는 숫자가 채워진 직육면체 형태로 해석할 수 있다.

In [21]:
x = np.array(
    [[[5, 78, 2, 34, 0], [6, 79, 3, 35, 1], [7, 80, 4, 36, 2]],
     [[5, 78, 2, 34, 0], [6, 79, 3, 35, 1], [7, 80, 4, 36, 2]],
     [[5, 78, 2, 34, 0], [6, 79, 3, 35, 1], [7, 80, 4, 36, 2]]]
)
x

array([[[ 5, 78,  2, 34,  0],
        [ 6, 79,  3, 35,  1],
        [ 7, 80,  4, 36,  2]],

       [[ 5, 78,  2, 34,  0],
        [ 6, 79,  3, 35,  1],
        [ 7, 80,  4, 36,  2]],

       [[ 5, 78,  2, 34,  0],
        [ 6, 79,  3, 35,  1],
        [ 7, 80,  4, 36,  2]]])

In [22]:
x.ndim

3

핵심

- 축, rank    : np.array(...).ndim
- 크기, shape : tensor의 축에 따라 얼마나 많은 차원이 있는지, np.array(...).shape
- 데이터 타입  : np.array(...).dtype
  - numpy와 여러 library들이 메모리 사전 할당 문제로 가변 길이의 문자열을 지원하지 않는다.
  - C언어도 아닌데 동적으로 짜면 느려진단 소리로 보인다.